# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Create our QandA application

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()  # needs OPENAI_API_KEY

from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

# NEW provider (works with openai==1.x)
from langchain_openai import OpenAIEmbeddings as NewOpenAIEmbeddings

# OLD base class that 0.0.312 checks against
from langchain.embeddings.base import Embeddings as OldEmbeddingsBase


class OpenAIEmbeddingsAdapter(OldEmbeddingsBase):
    """Adapter to make langchain_openai embeddings look like old LC embeddings."""
    def __init__(self, **kwargs):
        self._impl = NewOpenAIEmbeddings(**kwargs)

    def embed_documents(self, texts):
        return self._impl.embed_documents(texts)

    def embed_query(self, text):
        return self._impl.embed_query(text)


emb = OpenAIEmbeddingsAdapter(model="text-embedding-3-small")
# or "text-embedding-3-large" if you need higher quality

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=emb,
).from_loaders([loader])


In [8]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Coming up with test datapoints

In [9]:
data[10]

Document(page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10})

In [10]:
data[11]

Document(page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11})

### Hard-coded examples

In [11]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [12]:
from langchain.evaluation.qa import QAGenerateChain


In [21]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [14]:
# the warning below can be safely ignored

In [20]:
# --- Fix for LLMChain token_usage aggregation blowing up on OpenAIObject/etc. ---
import langchain.chains.llm as _lc_llm

if not hasattr(_lc_llm, "_APPLY_USAGE_PATCHED"):
    _orig_apply = _lc_llm.LLMChain.apply

    def _coerce_usage(u):
        # Convert OpenAIObject or other wrappers into plain dict(int)
        if hasattr(u, "to_dict"):
            u = u.to_dict()
        if isinstance(u, dict):
            fixed = {}
            for k, v in u.items():
                # unwrap nested OpenAIObject or similar
                if hasattr(v, "to_dict"):
                    v = v.to_dict()
                try:
                    fixed[k] = int(v)
                except Exception:
                    # last-ditch: try numeric fields or default to 0
                    try:
                        fixed[k] = int(float(v))
                    except Exception:
                        fixed[k] = 0
            return fixed
        # if it's already numeric-ish, just try to int it
        try:
            return int(u)
        except Exception:
            return 0

    def _patched_apply(self, input_list, callbacks=None):
        # Call original apply to run the batch
        result = _orig_apply(self, input_list, callbacks=callbacks)
        # Normalize each run's llm_output.token_usage to prevent += TypeError
        try:
            for r in result:
                lo = r.get("llm_output")
                if isinstance(lo, dict) and "token_usage" in lo:
                    lo["token_usage"] = _coerce_usage(lo["token_usage"])
        except Exception:
            # Be resilient; never fail here
            pass
        return result

    _lc_llm.LLMChain.apply = _patched_apply
    _lc_llm._APPLY_USAGE_PATCHED = True
# --- end patch ---


In [19]:
# new_examples = example_gen_chain.apply_and_parse(
#     [{"doc": t} for t in data[:5]]
# )

In [23]:
# new_examples = example_gen_chain.apply_and_parse(
#     [{"doc": t} for t in data[:5]])

In [31]:
new_examples = {'query': "What is the weight of each pair of Women's Campside Oxfords?",
 'answer': "The approximate weight of each pair of Women's Campside Oxfords is 1 lb. 1 oz."}


In [32]:
data[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

### Combine examples

In [33]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 ('page_content',
  ": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please 

In [34]:
examples += new_examples

In [36]:
# qa.run(examples[0]["query"])

## Manual Evaluation

In [37]:
import langchain
langchain.debug = True

In [39]:
# qa.run(examples[0]["query"])

In [ ]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [ ]:
predictions = qa.apply(examples)

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

In [ ]:
graded_outputs[0]

Reminder: Download your notebook to you local computer to save your work.